In [ ]:
#SWN20140227 Table: 0
#remaining bug: bcf in the first column...

In [13]:
%load_ext autoreload
%autoreload 2
import pickle
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from zlu_functions import del_subset_col, set_col_headers, \
                          row_header_to_col,remove_rows_for_total_value, \
                          dfs_need_transpose, remove_cols_for_fair_value, \
                          collar_fill_empty,remove_empty_rows_and_cols, \
                          df_column_uniquify, verticalSplit_instrument, \
                          verticalSplit_ZLU
from nel_functions import delete_uninformative,delete_empty,delete_duplicated, \
                          delete_subset,get_vertically_stacked_subtables, \
                          vertically_stacked_subtable_cleaner
from sja_functions import volumeCleaner, dateCleaner, priceCleaner,trimDictonary,verticalSplit, \
                   instrumentCleaner, productCleaner, filterBusiness,columnSorter, \
                   volumeCleaner_specialSauce
#pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')

pickle_in=open('hedge_dfs_sub_new.pickle', 'rb')
df_dic=pickle.load(pickle_in)

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_1 = delete_subset(df_dic)

ticker_list = []

for k in df_dic.keys():
    if k[:3] not in ticker_list:
        print(k[:3])
        ticker_list.append(k[:3])


df_dic = filterBusiness(df_dic, 'SWN')
# df_dic={'SWN20070301':df_dic['SWN20070301']}
#del df_dic['PKD20090302'] #false positive
with open('output_csv/output_before_spliting.csv','w') as f:
    pass
#preprocessing for all dfs
for k in df_dic.keys():
    for i in range(len(df_dic[k])):

        df=df_dic[k][i]
        df=set_col_headers(df) #set headers
        df=del_subset_col(df) #delete redundant cols
        df=remove_rows_for_total_value(df) #if any row for total value; save it
        df=remove_cols_for_fair_value(df)
        df_dic[k][i]=df
        with open('output_csv/output_before_spliting.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

#         print(k,'Original Table:',i,'-'*10)
#         display(HTML(df.to_html()))

#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    #print(k)
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    elif k=='BRY20110301':
        split_dfs = []
        for table in df_dic[k]:
            df1 = table.iloc[:, 0:3]
            df2 = table.iloc[:, 3:]
            split_dfs.append(df1)
            split_dfs.append(df2)
            df_dic[k]=split_dfs
    else: df_dic[k] = verticalSplit(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
 

for k in df_dic.keys():
    #print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=row_header_to_col(df)
            df=dfs_need_transpose(df)
            df=collar_fill_empty(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
#             print(k,'Table:',i,'-'*10,'After preprocessing')
#             display(HTML(df.to_html()))

#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    print(k)
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
    df_dic[k]=verticalSplit_instrument(df_dic[k])
    if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
    else: df_dic[k] = verticalSplit_ZLU(df_dic[k])    
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
           
#Horizontally Split
del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_3 = delete_subset(df_dic)

for k in df_dic.keys(): #apply Sam's spliting function
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
            
with open('output_csv/output_before_sam.csv','w') as f:
    pass
with open('output_csv/output_after_sam.csv','w') as f:
    pass
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('output_csv/output_before_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

date_keys = []               
date_stop = []

price_keys = ['weighted average floor price per mcf','weighted average price per mcf',
              'weighted average price per bbl', 'weighted average floor price per bbl',
              'weighted,average,price to be,swapped,($/mmbtu),,', 'weighted,average,floor,price,($/mmbtu),,',
              'weighted,average,ceiling,price,($/mmbtu),,', 'weighted,average,basis,differential,($/mmbtu),,',
              'weighted average price per mmbtu', 'weighted average price', 'weighted average floor price',
              'weighted average ceiling price', 'weighted average basis differential','call']
price_stop = []

volume_keys = ['(bcf)', '(mbbls)', 'volume']
volume_stop = ['natural gas (bcf)']               

#set verbose to True to turn on the diagonstic output for Sam's functions
verbose = False
key_override = True
                
for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
#         display(HTML(df.to_html()))
        if df.shape[0]>0: #apply Sam's functions to non-empty dfs
            print(k,'Table:',i,'-'*10,'Before Sam function')
            col_lst=[re.sub("[,]+","",col) if 'product_type' in col or 'instrument_type' in col else col for col in df.columns]
            df.columns=col_lst
            display(HTML(df.to_html()))
            df = instrumentCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = productCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = priceCleaner(df, key_words = price_keys, stop_words = price_stop,\
                                  verbose = verbose, key_override = key_override)
            df = volumeCleaner(df, key_words = volume_keys, stop_words = volume_stop,\
                                   verbose = verbose, key_override = key_override)
            df = dateCleaner(df, value_ids = ["20\d{2}"])
            #ZLU's change begins
            col_lst=[col for col in df.columns if 'vol' in col]
            for col in col_lst:
                df.loc[:,col]=df.loc[:,col].str.replace('[,\(\)]',"")
    #         display(HTML(df.to_html()))
    
    # build into functions volumeCleaner, row delete can stay here
            col_lst=['vol' in col for col in df.columns]
            row_lst=list(range(df.shape[0]))
            for row_idx in range(df.shape[0]):
                if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                    row_lst.remove(row_idx)
            df=df.iloc[row_lst]
            if all(df.loc[:,'vol_1_type']=='volume'): df.loc[:,'vol_1_type']=df.loc[:,'product_type'].str.extract('\(([^\)]+)\)')
            #ZLU's change ends
            
            print(k+' ,table:'+str(i)+'-----After Sam functions----')
            display(HTML(df.to_html()))
            print('~'*50)
        else: df=pd.DataFrame()
        df_dic[k][i]=df

del_dfs_dic_5 = delete_uninformative(df_dic)
print('1',len(df_dic))
delete_empty(df_dic)
print('2',len(df_dic))
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
print('3',len(df_dic))
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
print('4',len(df_dic))
del_dfs_dic_3 = delete_subset(df_dic)
print('5',len(df_dic))

print(len(df_dic.keys()))

big_df = pd.DataFrame()

for k in df_dic.keys():
    doc_id = k
    processed_dfs = df_dic[k]
    for df in processed_dfs:
        unprocessed_cols = columnSorter(df)
        print(unprocessed_cols)
        df.drop(unprocessed_cols, axis = 1, inplace = True)
             #zlu change starts
        col_lst=['price' in col for col in df.columns]
        row_lst=list(range(df.shape[0]))
        df['red_flag']=''
        for row_idx in range(df.shape[0]):
            if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                df.iloc[row_idx,-1]=1
        #zlu change ends

#         display(HTML(df.to_html()))
        length = df.shape[0]
        doc_id_col = np.array([doc_id]*length)
        df.insert(0, 'doc_id', doc_id_col)
        big_df = pd.concat([big_df, df], axis = 0, join = 'outer', sort = False)

print(big_df.shape)

if any(big_df['red_flag'] != ''):
    print("Check the red_flag column for missing values at")
    print('\n\n', big_df.iloc[big_df['red_flag'] != '', ].index)

try: big_df.to_csv('cleaned_csv/SWN_clean.csv', encoding='cp1252')
except: big_df.to_csv('cleaned_csv/SWN_clean.csv', encoding = 'utf-8')
# for k in df_dic.keys():
#     for i in range(len(df_dic[k])):
#         with open('APA_clean.csv','a') as f:
#             df=df_dic[k][i]
#             f.writelines(k+'\n'+'table:'+str(i))
#             try: df.to_csv(f,encoding='cp1252')
#             except: df.to_csv(f,encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
APA
APC
CLR
CPE
DVN
ECA
HES
MRO
MUR
NBL
OXY
PXD
RRC
SWN
UNT
SWN20030218
SWN20040225
SWN20050308
SWN20060306
SWN20070301
SWN20080228
SWN20090226
SWN20100225
SWN20110225
SWN20120228
SWN20130221
SWN20140227
SWN20150226
SWN20170223
SWN20030218
SWN20030218 Table: 0 ---------- Before Sam function


,additionalproduct_typeinstrument_type,"contract volume (bcf),production and marketing,natural gas,swaps with a fixed price receipt","weighted average price per mcf,production and marketing,natural gas,swaps with a fixed price receipt","contract amount (in millions),production and marketing,natural gas,swaps with a fixed price receipt"
1,"expected maturity date,,2003,,carrying,amount,",13.3,$3.47,$46.2
2,"expected maturity date,,2004,,carrying,amount,",7.2,$4.01,$28.8


SWN20030218 ,table:0-----After Sam functions----


,maturity,vol_1,vol_1_type,price_1,price_1_type,"contract amount (in millions),production and marketing,natural gas,swaps with a fixed price receipt",instrument_type,product_type
1,2003,13.3,bcf,$3.47,weighted average price,$46.2,swap,natural gas
2,2004,7.2,bcf,$4.01,weighted average price,$28.8,swap,natural gas


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SWN20030218 Table: 1 ---------- Before Sam function


...

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SWN20170223
SWN20170223 Table: 0 ---------- Before Sam function


,",",",volume (bcf)","weighted average price per mmbtu,swaps","weighted average price per mmbtu,sold puts","weighted average price per mmbtu,purchased puts","weighted average price per mmbtu,sold calls","weighted average price per mmbtu,basis differential",additional,instrument_type
4,Fixed price swaps,322,$3.07,,,,,2017,
5,Two-way costless collars,103,,,2.94,3.38,,2017,
6,Three-way costless collars,135,,2.29,2.97,3.30,,2017,
7,Basis swaps,132,,,,,Neg0.87,2017,
10,Fixed price swaps,18,$3.00,,,,,2018,
11,Two-way costless collars,14,,,3.00,3.46,,2018,
12,Three-way costless collars,208,,2.37,2.96,3.37,,2018,
13,Basis swaps,16,,,,,Neg0.94,2018,
16,Three-way costless collars,62,,$2.50,$2.92,$3.35,,2019,
20,2017,86,,,,$3.25,,2019,sold call options


instrument_type column already exists
SWN20170223 ,table:0-----After Sam functions----


,maturity,vol_1,vol_1_type,price_1,price_1_type,price_2,price_2_type,price_3,price_3_type,price_4,price_4_type,price_5,price_5_type,additional,instrument_type,product_type
4,2017,322,bcf,$3.07,weighted average price,,weighted average price,,weighted average price,,call,,weighted average price,2017,swap,@
5,2017,103,bcf,,weighted average price,,weighted average price,2.94,weighted average price,3.38,call,,weighted average price,2017,collar,@
6,2017,135,bcf,,weighted average price,2.29,weighted average price,2.97,weighted average price,3.30,call,,weighted average price,2017,collar,@
7,2017,132,bcf,,weighted average price,,weighted average price,,weighted average price,,call,Neg0.87,weighted average price,2017,basis swap,natural gas
10,2018,18,bcf,$3.00,weighted average price,,weighted average price,,weighted average price,,call,,weighted average price,2018,swap,@
11,2018,14,bcf,,weighted average price,,weighted average price,3.00,weighted average price,3.46,call,,weighted average price,2018,collar,@
12,2018,208,bcf,,weighted average price,2.37,weighted average price,2.96,weighted average price,3.37,call,,weighted average price,2018,collar,@
13,2018,16,bcf,,weighted average price,,weighted average price,,weighted average price,,call,Neg0.94,weighted average price,2018,basis swap,natural gas
16,2019,62,bcf,,weighted average price,$2.50,weighted average price,$2.92,weighted average price,$3.35,call,,weighted average price,2019,collar,@
20,2017,86,bcf,,weighted average price,,weighted average price,,weighted average price,$3.25,call,,weighted average price,2019,sold call options,@


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 14
2 14
3 14
4 14
5 14
14
['contract amount (in millions),production and marketing,natural gas,swaps with a fixed price receipt']
['contract amount of floor (in millions),production and marketing,natural gas,price collars', 'contract amount of ceiling (in millions),production and marketing,natural gas,price collars']
['contract amount (in millions),production and marketing,oil,swaps with a fixed price receipt']
['contract amount (in millions),production and marketing,natural gas purchases,swaps with a fixed price payment']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
(100, 18)


In [12]:
big_df.columns

Index(['doc_id', 'maturity', 'vol_1', 'vol_1_type', 'price_1', 'price_1_type',
       'instrument_type', 'product_type', 'red_flag', 'price_2',
       'price_2_type', 'price_3', 'price_3_type', 'price_4', 'price_4_type',
       'price_5', 'price_5_type', 'additional'],
      dtype='object')

In [7]:
re.search('\d', '-')